Dataset
==

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
#load needed packages
import pandas as pd
import random 
import os

In [3]:
#make directory for data 
DATA_PATH = 'data/'
os.makedirs(DATA_PATH, exist_ok=True)
RAW_DATA = f'{DATA_PATH}raw_data/'
os.makedirs(RAW_DATA, exist_ok=True)

## 1. Loading Dataset


### 1.1. Download Dataset
First we download and extract the gigaword dataset (~3M) [here](https://drive.google.com/file/d/0B6N7tANPyVeBNmlSX19Ld2xDU1E/view?usp=sharing)

Meantime using the BBC dataset (7M) here: https://www.kaggle.com/pariza/bbc-news-summary/download

In [9]:
#download data (CNN)
# !curl --header "Host: doc-0c-3o-docs.googleusercontent.com" --header "User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header "Accept-Language: en-US,en;q=0.9" --header "Cookie: AUTH_s4a6oitvorbtivfcjm2pefm907l1ntir=08690680304265769485|1531648800000|pm1jthdhng09cikkb0pkdcqlod4d76p8" --header "Connection: keep-alive" "https://doc-0c-3o-docs.googleusercontent.com/docs/securesc/d1n9duui70mcvt9ph3953bv4foh1d3fm/pb1h3k6beg14nfv16poorm9mr6bl90e3/1531656000000/03129501499031348422/08690680304265769485/0B6N7tANPyVeBNmlSX19Ld2xDU1E?e=download" -o "summary.tar.gz" -L



In [7]:
#extract files 
!tar -xzf summary.tar.gz -C {RAW_DATA} && mv summary.tar.gz {RAW_DATA}
!gunzip {RAW_DATA}sumdata/train/*.*.txt.gz 

tar: Error opening archive: Unrecognized archive format
zsh:1: no matches found: data/raw_data/sumdata/train/*.*.txt.gz


### 1.2. Import Datasets

In [ ]:
/Users/frankkelly/Dropbox/Projects-new/abstractive-text-summarization/data/raw_data/BBC News Summary/BBC News Summary/Summaries/business

In [16]:
pd.read_csv(f'{RAW_DATA}BBC News Summary/BBC News Summary/News Articles/business/001.txt', sep="\n")

,Ad sales boost Time Warner profit
0,Quarterly profits at US media giant TimeWarner...
1,"The firm, which is now one of the biggest inve..."
2,Time Warner said on Friday that it now owns 8%...
3,Time Warner's fourth quarter profits were slig...
4,TimeWarner is to restate its accounts as part ...


In [27]:
def read_text_blob_in(path:str=f'{RAW_DATA}BBC News Summary/BBC News Summary/Summaries/business/001.txt') -> str:
    with open(path) as file:
        return file.read()
        
    

In [32]:
raw_article_string = read_text_blob_in(f'{RAW_DATA}BBC News Summary/BBC News Summary/News Articles/business/001.txt')
raw_article_string

'Ad sales boost Time Warner profit\n\nQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.\n\nThe firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.\n\nTime Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL\'s underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sig

In [33]:
raw_summary_string = read_text_blob_in(f'{RAW_DATA}BBC News Summary/BBC News Summary/Summaries/business/001.txt')
raw_summary_string

"TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn.Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters.Time Warner's fourth quarter profits were slightly better than analysts' expectations."

In [9]:
#join train sentence pairs together into dataframe
# data = pd.concat([pd.read_csv(f'{RAW_DATA}sumdata/train/train.article.txt', sep="\n"), 
#                   pd.read_csv(f'{RAW_DATA}sumdata/train/train.title.txt', sep="\n")], axis=1)
# data.columns = ["article", "title"]


In [10]:
#join validation sentence pairs together into dataframe
val = pd.concat([pd.read_csv(f'{RAW_DATA}sumdata/train/valid.article.filter.txt', sep="\n"), 
                  pd.read_csv(f'{RAW_DATA}sumdata/train/valid.title.filter.txt', sep="\n")], axis=1)
val.columns = ["article", "title"]


In [10]:
#join validation sentence pairs together into dataframe
# val = pd.concat([pd.read_csv(f'{RAW_DATA}sumdata/train/valid.article.filter.txt', sep="\n"), 
#                   pd.read_csv(f'{RAW_DATA}sumdata/train/valid.title.filter.txt', sep="\n")], axis=1)
# val.columns = ["article", "title"]


In [13]:
len(data), len(val)

(3803956, 189650)

## 2. Preprocessing Dataset

### 2.1. Looking at the data

In [26]:
for i in range(5):
    r = random.randint(0,50)
    display(data.iloc[r]['article'])
    display(data.iloc[r]['title'])


'new zealand share prices closed #.## percent higher monday in subdued trading ahead of a us holiday , dealers said .'

'new zealand stocks close #.## percent higher'

"kenyan police have a mounted a ##-hour patrol in a village where us presidential candidate barack obama 's grandmother lives after robbers made a botched attempt to rob her solar panel , an official said ."

"kenyan police offer obama 's grandmother security after robbery"

'an israeli military helicopter with a two-man crew crashed near the northern town of afula and burst into flames on wednesday , army radio said .'

'military helicopter crashes in israel'

"bolivian president evo morales on wednesday declared the us ambassador to la paz `` persona non grata , '' accusing the envoy of encouraging the breakup of the country by promoting separatism ."

'bolivia president orders us envoy expelled'

'the algerian cabinet chaired by president abdelaziz bouteflika on sunday adopted the #### finance bill predicated on an oil price of ## dollars a barrel and a growth rate of #.# percent , it was announced here .'

'algeria adopts #### finance bill with oil put at ## dollars a barrel'

### Save data in tabular format

In [11]:
PROCESSED_DATA_PATH = f'{DATA_PATH}processed_data/'
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

In [12]:
#save train, val datasets 
data.to_csv(f'{PROCESSED_DATA_PATH}train_ds.csv', index=None)
val.to_csv(f'{PROCESSED_DATA_PATH}valid_ds.csv', index=None)

### Create Sample Data

In [5]:
SAMPLE_DATA_PATH = f'{DATA_PATH}sample_data/'
os.makedirs(SAMPLE_DATA_PATH, exist_ok=True)

In [6]:
sample_train = data.sample(80000)
sample_val = val.sample(20000)

In [7]:
len(sample_train), len(sample_val)

(80000, 20000)

In [8]:
#save sample train, val, and test datasets 
sample_train.to_csv(f'{SAMPLE_DATA_PATH}train_ds.csv', index=None)
sample_val.to_csv(f'{SAMPLE_DATA_PATH}valid_ds.csv', index=None)

### Create Smaller Sample Data

In [9]:
sample_train_ = data.sample(64)
sample_val_ = val.sample(16)

In [10]:
len(sample_train_), len(sample_val_)

(64, 16)

In [11]:
#save sample train, val, and test datasets 
sample_train_.to_csv(f'{SAMPLE_DATA_PATH}train_ds_.csv', index=None)
sample_val_.to_csv(f'{SAMPLE_DATA_PATH}valid_ds_.csv', index=None)

In [2]:
import pandas as pd
DATA_PATH = 'data/'
PROCESSED_DATA_PATH = f'{DATA_PATH}processed_data/'
data = pd.read_csv(f'{PROCESSED_DATA_PATH}train_ds.csv')
val = pd.read_csv(f'{PROCESSED_DATA_PATH}valid_ds.csv')
